In [20]:
import pandas as pd
import numpy as np
df = pd.read_csv("../data/input.csv")
df.head()

,Unnamed: 0,title,img_captions_str,meme_captions_str,metaphors_str,url,input
0,0,For real though,Person in Spider Man outfit gives a lecture on...,Meme poster is frustrated about the format of ...,"Spider Man outfit: Meme poster, a lecture: com...",https://i.redd.it/m16dhaqyply21.jpg,TEXT: For real though; Person in Spider Man ou...
1,1,And that's a fact,"Two dogs carry a white flag, Identical chubby ...",Meme poster is saying that searching Google pl...,"Two dogs: Meme poster, a white flag: PSA, carr...",https://i.redd.it/z9oh7ligb0i31.jpg,TEXT: And that's a fact; Two dogs carry a whit...
2,2,It was horrible,"man is very dissatisfied with what he sees, Qu...",Meme poster is feeling sad.,"man: Meme poster, sees: expressing different r...",https://i.redd.it/yves3izsbsj31.jpg,TEXT: It was horrible; man is very dissatisfie...
3,3,This is why Reddit is better,A man that is crying and a man that is laughin...,Meme poster is saying that Reddit is more fun ...,"A man: Meme poster, a man: Meme poster, his go...",https://i.redd.it/y594n8exi6k31.jpg,TEXT: This is why Reddit is better; A man that...
4,4,The Area 51 raid is still happening right?,"a cop is chasing a thief with a shopping cart,...",Meme poster is making fun of the Area 51 raid.,"a cop: Meme poster, a thief: not being bothere...",https://i.redd.it/4hrn18t4lck31.jpg,TEXT: The Area 51 raid is still happening righ...


In [3]:
from transformers import AutoTokenizer,AutoModel

model_cpkt = "sentence-transformers/multi-qa-mpnet-base-dot-v1"
tokenizer = AutoTokenizer.from_pretrained(model_cpkt)
model = AutoModel.from_pretrained(model_cpkt)

In [4]:
import torch

device = torch.device("mps")
model.to(device)

MPNetModel(
  (embeddings): MPNetEmbeddings(
    (word_embeddings): Embedding(30527, 768, padding_idx=1)
    (position_embeddings): Embedding(514, 768, padding_idx=1)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): MPNetEncoder(
    (layer): ModuleList(
      (0-11): 12 x MPNetLayer(
        (attention): MPNetAttention(
          (attn): MPNetSelfAttention(
            (q): Linear(in_features=768, out_features=768, bias=True)
            (k): Linear(in_features=768, out_features=768, bias=True)
            (v): Linear(in_features=768, out_features=768, bias=True)
            (o): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (intermediate): MPNetIntermediate(
          (dense): Linear(in_

In [5]:
def cls_pooling(model_output):
    return model_output.last_hidden_state[:, 0]

In [6]:
def get_embeddings(text_list):
    encoded_input = tokenizer(
        text_list, padding=True, truncation=True, return_tensors="pt"
    )
    encoded_input = {k: v.to(device) for k, v in encoded_input.items()}
    model_output = model(**encoded_input)
    return cls_pooling(model_output)

In [7]:
from datasets import Dataset

In [8]:
dataset = Dataset.from_pandas(df)

In [9]:
dataset

Dataset({
    features: ['Unnamed: 0', 'title', 'img_captions_str', 'meme_captions_str', 'metaphors_str', 'url', 'input'],
    num_rows: 5818
})

In [9]:
embedding = get_embeddings(dataset["input"][0])
embedding.shape

torch.Size([1, 768])

In [17]:
embeddings_dataset = dataset.map(
    lambda x: {"embeddings": get_embeddings(x["input"]).detach().cpu()[0]}
)

  0%|          | 0/5818 [00:00<?, ?ex/s]

In [22]:
embeddings_dataset = embeddings_dataset.with_format("np")

In [25]:
embeddings_dataset.add_faiss_index(column="embeddings")

AttributeError: module 'numpy' has no attribute 'object'.
`np.object` was a deprecated alias for the builtin `object`. To avoid this error in existing code, use `object` by itself. Doing this will not modify any behavior and is safe. 
The aliases was originally deprecated in NumPy 1.20; for more details and guidance see the original release note at:
    https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations

In [ ]:
query = "Spiderman giving lecture"
question_embedding = get_embeddings([question]).cpu().detach()
question_embedding.shape